In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import seaborn as sns
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

In [2]:
df = pd.read_csv('tagged_tweets.csv')
df.hashtags = df.hashtags.apply(eval)
df.head()

,id,text,non_hashtags_text,author_id,hashtags
0,1512581807853633541,RT @forever_seesaw: #PTDinLV Do not Uber/Lyft ...,RT @forever_seesaw: <UNKTAG> Do not Uber/Lyft ...,162180146,[PTDinLV]
1,1512581787075026944,RT @FOX5Vegas: #BTS mania has completely taken...,RT @FOX5Vegas: <UNKTAG> mania has completely t...,952923842363645952,[BTS]
2,1512581771153461248,RT @FOX5Vegas: #BTS mania has completely taken...,RT @FOX5Vegas: <UNKTAG> mania has completely t...,1370022606632484867,[BTS]
3,1512581733543120896,Army walk to stadiam. 💜 คนเดินข้ามสะพานเพื่อมา...,Army walk to stadiam. 💜 คนเดินข้ามสะพานเพื่อมา...,1285244583228502017,[PTD_ON_STAGE_LV]
4,1512581724718661632,RT @URDailyHistory: 8 April 1974: #Atlanta Bra...,RT @URDailyHistory: 8 April 1974: <UNKTAG> Bra...,1126271645000163328,"[Atlanta, outfielder, Aaron]"


In [3]:
for i in tqdm(range(len(df))):
    df['hashtags'].loc[i] = df['hashtags'].loc[i][0]

df.head()

  0%|          | 0/34980 [00:00<?, ?it/s]C:\Users\10213\AppData\Local\Temp/ipykernel_28404/3716890117.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hashtags'].loc[i] = df['hashtags'].loc[i][0]
100%|██████████| 34980/34980 [04:37<00:00, 125.85it/s]


,id,text,non_hashtags_text,author_id,hashtags
0,1512581807853633541,RT @forever_seesaw: #PTDinLV Do not Uber/Lyft ...,RT @forever_seesaw: <UNKTAG> Do not Uber/Lyft ...,162180146,PTDinLV
1,1512581787075026944,RT @FOX5Vegas: #BTS mania has completely taken...,RT @FOX5Vegas: <UNKTAG> mania has completely t...,952923842363645952,BTS
2,1512581771153461248,RT @FOX5Vegas: #BTS mania has completely taken...,RT @FOX5Vegas: <UNKTAG> mania has completely t...,1370022606632484867,BTS
3,1512581733543120896,Army walk to stadiam. 💜 คนเดินข้ามสะพานเพื่อมา...,Army walk to stadiam. 💜 คนเดินข้ามสะพานเพื่อมา...,1285244583228502017,PTD_ON_STAGE_LV
4,1512581724718661632,RT @URDailyHistory: 8 April 1974: #Atlanta Bra...,RT @URDailyHistory: 8 April 1974: <UNKTAG> Bra...,1126271645000163328,Atlanta


In [4]:
df.hashtags.value_counts()

BTS                461
Ukraine            342
Tigray             335
PTD_ON_STAGE_LV    288
BBMAs              226
                  ... 
Guardian             1
Protospiel           1
ワンインチチャンネル           1
위아이                  1
Dreamcatcher         1
Name: hashtags, Length: 7062, dtype: int64

In [5]:
vocabulary = {}
decoder = {}
frequency = {}
cnt = 0
for tags in tqdm(df.hashtags.value_counts().index):
    if df.hashtags.value_counts()[tags] > 20:
        frequency[tags] = df.hashtags.value_counts()[tags]
        vocabulary[tags] = cnt
        decoder[cnt] = tags
        cnt += 1
df['class'] = -1
for i in tqdm(range(len(df))):
    if df['hashtags'].loc[i] in vocabulary:
        df['class'].loc[i] = vocabulary[df['hashtags'].loc[i]]




  0%|          | 0/34980 [00:00<?, ?it/s]C:\Users\10213\AppData\Local\Temp/ipykernel_28404/236368660.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['class'].loc[i] = vocabulary[df['hashtags'].loc[i]]
100%|██████████| 34980/34980 [01:31<00:00, 383.74it/s]


In [6]:
df.head()

,id,text,non_hashtags_text,author_id,hashtags,class
0,1512581807853633541,RT @forever_seesaw: #PTDinLV Do not Uber/Lyft ...,RT @forever_seesaw: <UNKTAG> Do not Uber/Lyft ...,162180146,PTDinLV,-1
1,1512581787075026944,RT @FOX5Vegas: #BTS mania has completely taken...,RT @FOX5Vegas: <UNKTAG> mania has completely t...,952923842363645952,BTS,0
2,1512581771153461248,RT @FOX5Vegas: #BTS mania has completely taken...,RT @FOX5Vegas: <UNKTAG> mania has completely t...,1370022606632484867,BTS,0
3,1512581733543120896,Army walk to stadiam. 💜 คนเดินข้ามสะพานเพื่อมา...,Army walk to stadiam. 💜 คนเดินข้ามสะพานเพื่อมา...,1285244583228502017,PTD_ON_STAGE_LV,3
4,1512581724718661632,RT @URDailyHistory: 8 April 1974: #Atlanta Bra...,RT @URDailyHistory: 8 April 1974: <UNKTAG> Bra...,1126271645000163328,Atlanta,-1


In [7]:
decoder[1]

'Ukraine'

In [8]:
data = df[df["class"] != -1].dropna()

In [9]:
data.head()

,id,text,non_hashtags_text,author_id,hashtags,class
1,1512581787075026944,RT @FOX5Vegas: #BTS mania has completely taken...,RT @FOX5Vegas: <UNKTAG> mania has completely t...,952923842363645952,BTS,0
2,1512581771153461248,RT @FOX5Vegas: #BTS mania has completely taken...,RT @FOX5Vegas: <UNKTAG> mania has completely t...,1370022606632484867,BTS,0
3,1512581733543120896,Army walk to stadiam. 💜 คนเดินข้ามสะพานเพื่อมา...,Army walk to stadiam. 💜 คนเดินข้ามสะพานเพื่อมา...,1285244583228502017,PTD_ON_STAGE_LV,3
5,1512581722931671042,RT @Army_Connect: First thing we saw upon arri...,RT @Army_Connect: First thing we saw upon arri...,1079502652835282944,PTD_ON_STAGE_LV,3
6,1512581703298334720,RT @sunflowercharts: .@BTS_twt เตรียมจัดคอนเสิ...,RT @sunflowercharts: .@BTS_twt เตรียมจัดคอนเสิ...,1270265403739004929,BTS,0


In [10]:
len(data)

11793

In [11]:
import random
drop_rows = []
num_totalTag = len(data)
text_list = []
tag_list = []
class_list = []
for index, row in tqdm(data.iterrows()):
    # sampling
    z = frequency[row[-2]] / num_totalTag
    p = ((z / 0.001)**0.5 + 1) * (0.001 / z)
    for j in range(10):
        outcome = random.random() # a random number from 0 to 1.
                
        if outcome <= p:
            text_list.append(row[-4])
            tag_list.append(row[-2])
            class_list.append(row[-1])

11793it [00:00, 13664.59it/s]


In [12]:
data2 = pd.DataFrame({"non_hashtags_text":text_list,"hashtags":tag_list,"class":class_list})
data2.hashtags.value_counts()

BTS                 850
Ukraine             782
Tigray              696
Bitcoin             672
BBMAs               661
                   ... 
Justiceforbella     210
quote               210
Wishlist            210
Siguemeytesigo      210
SonicTheHedgehog    210
Name: hashtags, Length: 210, dtype: int64

In [13]:
len(data2)

71561

In [14]:
from sklearn.model_selection import train_test_split
# ----- 1. Preprocess data -----#
# Preprocess data
X = list(data2["non_hashtags_text"])
y = list(data2["class"])
# X_val = list(data["non_hashtags_text"][10000:11000])
# y_val = list(data["class"][10000:11000])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
X_val = X_test[:1000]
y_val = y_test[:1000]

In [26]:
# tokenizer from hugging face
from transformers import AutoTokenizer, BertTokenizer, BertModel
from transformers import AutoModelForMaskedLM, BertForSequenceClassification


# development = dev_df.dropna()

# Define pretrained tokenizer and model
# "vinai/bertweet-base",
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", use_fast=False)
tokenizer.mask_token = '<UNKTAG>'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(vocabulary))





X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=20)
# y_train_tokenized = tokenizer(y_train, padding=True, truncation=True, max_length=50)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=20)   
# y_val_tokenized = tokenizer(y_val, padding=True, truncation=True, max_length=50)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at C:\Users\10213/.cache\huggingface\transformers\3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/r

In [32]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:    
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

train_dataset.__getitem__(9)

{'input_ids': tensor([  101,  1030, 26319, 12199, 11431, 12184,  1045,   100,  2005,  5440,
          2931,  3063,  2012,  1996,   100,  1052,   102,     0,     0,     0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]),
 'labels': tensor(62)}

In [33]:
# ----- 2. Fine-tune pretrained model -----#
# Define Trainer parameters
from sklearn.metrics import mean_squared_error, f1_score
from transformers import EarlyStoppingCallback, TrainingArguments, Trainer

# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     rmse = mean_squared_error(labels, predictions, squared=False)
#     return {"rmse": rmse}


def compute_metrics(p):    
    pred, labels = p
    pred = np.argmax(pred, axis=1)
#     accuracy = accuracy_score(y_true=labels, y_pred=pred)
#     recall = recall_score(y_true=labels, y_pred=pred)
#     precision = precision_score(y_true=labels, y_pred=pred)
#    f1 = f1_score(y_true=labels, y_pred=pred)
    f1_mac = f1_score(y_true=labels, y_pred=pred, average="macro")
    return {"f1_macro": f1_mac}



# Define Trainer
args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=1,
    seed=0,
    load_best_model_at_end=True,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=10)],
)

# Train pre-trained model
trainer.train()


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

'not'

In [200]:
X = tokenizer(X_test[1000:], padding=True, truncation=True, max_length=20)
Xevaluation = Dataset(X)


In [34]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
raw_pred, _, _ = trainer.predict(Xevaluation)
# pred = raw_pred.squeeze(1)



RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [202]:
pred = np.argmax(raw_pred, axis=1)

In [203]:
pred

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [161]:
from transformers import pipeline
task = pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer
)
task("Hello world")[0]['label']

'LABEL_4'

In [1]:
res = []
for i in pred:
    
    res.append(decoder[i])
res

NameError: name 'pred' is not defined

In [ ]:
# most frequent


In [205]:
y_true = []
for i in range(len(y_test[1000:])):
    y_true.append(decoder[y_test[1000:][i]])

In [177]:
y_true

['COVID19',
 'Sukhbaatar',
 'SBM2022',
 'Jimin',
 'DAY6',
 'GONZAGA',
 'IndieGameDev',
 'ToEvangeliseChrist',
 'Fear',
 'Crypto',
 'Rockets',
 'listing',
 'EXO',
 'OppositionHypocrisyExposed',
 'رمضان_مميز_بايه',
 'ScottyTheWorstPrimeMinisterEver',
 'ICOMCodeOfEthics',
 'TigerWoods',
 'teamchais',
 'fat',
 'Fukushima',
 'Binance',
 'BTS',
 'AFLWGF',
 'KetanjiBrownJackson',
 'Tigray',
 'SaturdayVibes',
 'HelluvaBoss',
 'Ukraine',
 'Amazonギフト券',
 'رمضان_مميز_بايه',
 'StandingOnaBox',
 'impressionism',
 'هدايا_ستايلش',
 'DeFi',
 'BuildWealth',
 'TVD',
 'MNTwins',
 'BNX',
 'carhire',
 'WorldPremiere',
 'BBN',
 'Mayabeque',
 'StandUpForUkraine',
 'Bitcoin',
 'Essay',
 'Bajwa',
 'Kramatorsk',
 'comm_college',
 '羅德弘',
 'CapFoodFight',
 'Dogging',
 'nj',
 'Airdrop',
 'ZLAN2022',
 'stlblues',
 'PP22000093736',
 'SexRol',
 'avgeeks',
 'FuckTexas',
 'France',
 'YOONJAEHYUK',
 'EXO',
 'DonBelle',
 'UnDíaComoHoy',
 'Breakthesilence',
 'FerrariDaytonaSP3',
 'SmackDown',
 'StayAlive',
 'Advertising',

In [172]:
w2v_mod = KeyedVectors.load_word2vec_format('your-vector-file-debiased.kv', binary=False)

In [173]:
def cos_sim(predict, actual):
    result = 0
    mag_pre = 1
    mag_act = 1
    for token in actual:
        if predict[0] == token:
            result = 1
        else:
            try:
                predict_0 = w2v_mod[predict[0]]
                mag_pre = np.linalg.norm(predict_0)
            except:
                predict_0 = np.zeros(100)
                mag_pre = 1
            try:
                actual_i = w2v_mod[token]
                mag_act = np.linalg.norm(actual_i)
            except:
                actual_i = np.zeros(100)
                mag_act = 1
            result = max(result, (sum(np.multiply(predict_0, actual_i)) / mag_pre / mag_act))

    return result

In [206]:
avg_score = 0
for i in range(len(res)):
    
    avg_score += cos_sim([res[i]], [y_true[i]])
avg_score /= len(res)
avg_score

0.0027795595467179817